In [ ]:
##Advanced Network Programming
import socket
import select

class NonBlockingServer:
    def __init__(self, host='localhost', port=8888):
        self.server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.server_socket.bind((host, port))
        self.server_socket.listen(5)
        self.server_socket.setblocking(False)
        self.inputs = [self.server_socket]
        self.outputs = []
        self.message_queues = {}

    def run(self):
        while self.inputs:
            readable, writable, exceptional = select.select(
                self.inputs, self.outputs, self.inputs)
            
            for s in readable:
                if s is self.server_socket:
                    connection, client_address = s.accept()
                    connection.setblocking(False)
                    self.inputs.append(connection)
                    self.message_queues[connection] = []
                else:
                    data = s.recv(1024)
                    if data:
                        self.message_queues[s].append(data)
                        if s not in self.outputs:
                            self.outputs.append(s)
                    else:
                        if s in self.outputs:
                            self.outputs.remove(s)
                        self.inputs.remove(s)
                        s.close()
                        del self.message_queues[s]
